# Introdução

## Importação das bibliotecas necessarias

In [181]:
%matplotlib inline
import requests
import pandas as pd
import os
import json
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import random as random
import numpy as np
import ipywidgets as widgets
import glob, os
sns.set()

## Importação das tabelas

In [151]:
df_2015 = pd.read_csv('../csv_files/rmm/nascidos_vivos_2015.csv', sep=';')
df_2016 = pd.read_csv('../csv_files/rmm/nascidos_vivos_2016.csv', sep=';')
df_2017 = pd.read_csv('../csv_files/rmm/nascidos_vivos_2017.csv', sep=';')
df_2018 = pd.read_csv('../csv_files/rmm/nascidos_vivos_2018.csv', sep=';')
df_2019 = pd.read_csv('../csv_files/rmm/nascidos_vivos_2019.csv', sep=';')
df_rmm = pd.read_csv('../csv_files/QW-20-razao_mortalidade_materna.csv')
df_rmm_direta = pd.read_csv('../csv_files/QW-21-razao_mortalidade_materna_direta.csv')

## Ajuste dos anos de 2015 a 2019 a partir Estimativa do IBGE

In [77]:
df_rmm.loc[(df_rmm['CODMUNRES'].isin([330455]))]

,CODMUNRES,Ano,Qtd_Morte_Materna,Qtd_Total,Ratio
26031,330455,2012,59.0,86549.0,68.169476
26032,330455,2013,67.0,87661.0,76.430796
26033,330455,2014,64.0,89991.0,71.118223
26034,330455,2015,65.0,90694.0,71.669570
26035,330455,2016,62.0,83166.0,74.549696
26036,330455,2017,70.0,84521.0,82.819654
26037,330455,2018,50.0,82554.0,60.566417
26038,330455,2019,62.0,76803.0,80.726013


In [78]:
df_rmm_direta.loc[(df_rmm['CODMUNRES'].isin([330455]))]

,CODMUNRES,Ano,Qtd_Morte_Materna_Direta,Qtd_Total,Ratio
26031,330455,2012,36.0,86549,41.594935
26032,330455,2013,35.0,87661,39.926535
26033,330455,2014,32.0,89991,35.559111
26034,330455,2015,45.0,90694,49.617395
26035,330455,2016,32.0,83166,38.477262
26036,330455,2017,40.0,84521,47.325517
26037,330455,2018,26.0,82554,31.494537
26038,330455,2019,34.0,76803,44.269104


In [83]:
#df_rmm
df_2015['Código Município'] = df_2015['Código Município'].astype(str)
df_2016['Código Município'] = df_2016['Código Município'].astype(str)
df_2017['Código Município'] = df_2017['Código Município'].astype(str)
df_2018['Código Município'] = df_2018['Código Município'].astype(str)
df_2019['Código Município'] = df_2019['Código Município'].astype(str)

df_rmm['CODMUNRES'] = df_rmm['CODMUNRES'].astype(str)
df_rmm_direta['CODMUNRES'] = df_rmm_direta['CODMUNRES'].astype(str)

for i in range(2015, 2020):
    for index, row in globals()['df_'+str(i)].iterrows():
        if(len(row[0]) == 7):
            mun = row[0][:-1]
        else:
            mun = row[0]
        idx = df_rmm.loc[(df_rmm['CODMUNRES'].isin([mun])) & (df_rmm['Ano'] == i)].index[0]
        df_rmm.at[idx, 'Qtd_Total']=row[1]
        idx = df_rmm_direta.loc[(df_rmm_direta['CODMUNRES'].isin([mun])) & (df_rmm_direta['Ano'] == i)].index[0]
        df_rmm_direta.at[idx, 'Qtd_Total']=row[1]
    print('foi '+str(i))
print('foi tudo')


foi 2015
foi 2016
foi 2017
foi 2018
foi 2019
foi tudo


In [88]:
df_rmm['Ratio'] = ((df_rmm['Qtd_Morte_Materna']/df_rmm['Qtd_Total'])*100000)
df_rmm.loc[(df_rmm['CODMUNRES'].isin(['330455']))]

,CODMUNRES,Ano,Qtd_Morte_Materna,Qtd_Total,Ratio
26031,330455,2012,59.0,86549.0,68.169476
26032,330455,2013,67.0,87661.0,76.430796
26033,330455,2014,64.0,89991.0,71.118223
26034,330455,2015,65.0,96059.0,67.666746
26035,330455,2016,62.0,84638.0,73.253149
26036,330455,2017,70.0,85132.0,82.225250
26037,330455,2018,50.0,83138.0,60.140970
26038,330455,2019,62.0,77137.0,80.376473


In [89]:
df_rmm_direta['Ratio'] = ((df_rmm_direta['Qtd_Morte_Materna_Direta']/df_rmm_direta['Qtd_Total'])*100000)
df_rmm_direta.loc[(df_rmm_direta['CODMUNRES'].isin(['330455']))]

,CODMUNRES,Ano,Qtd_Morte_Materna_Direta,Qtd_Total,Ratio
26031,330455,2012,36.0,86549,41.594935
26032,330455,2013,35.0,87661,39.926535
26033,330455,2014,32.0,89991,35.559111
26034,330455,2015,45.0,96059,46.846209
26035,330455,2016,32.0,84638,37.808077
26036,330455,2017,40.0,85132,46.985857
26037,330455,2018,26.0,83138,31.273305
26038,330455,2019,34.0,77137,44.077421


In [91]:
df_rmm.to_csv('../csv_files/QW-20C-razao_mortalidade_materna.csv',index=False)
df_rmm_direta.to_csv('../csv_files/QW-21C-razao_mortalidade_materna_direta.csv',index=False)

## Sorteamento dos municípios pelas faixas

In [182]:
df_estados = pd.read_csv('../csv_files/Estados.csv')
df_municipios = pd.read_csv('../csv_files/Municipios.csv')
df_pop_fem_10_49 = pd.read_csv('../tabelas_tabnet/mulheres_10_49_por_municipio_limpa.csv', ';')

df_rmm = pd.read_csv('../csv_files/QW-20C-razao_mortalidade_materna.csv')
df_rmm_direta = pd.read_csv('../csv_files/QW-21C-razao_mortalidade_materna_direta.csv')

In [183]:

which_df = 'df_rmm_direta'
#globals()[which_df]

coluna_codigo = globals()[which_df].columns[0] 
ano =           globals()[which_df].columns[1] 
indicador =     globals()[which_df].columns[2]
total =         globals()[which_df].columns[3]
ratio =         globals()[which_df].columns[4]
print(coluna_codigo,ano,indicador,total,ratio)

years = []
for col in df_pop_fem_10_49:
    if col != 'Municipio':
        years.append(col)

all_entries = []
for index, row in df_pop_fem_10_49.iterrows():
    row_array = row.values.tolist()
    for i in range(len(row_array)):
        if i != 0:
            pop = row_array[i]
            if pop == '-':
                pop = row_array[i+1]
            all_entries.append([row_array[0], years[i-1], int(pop)])

            
df_pop_fem_10_49_by_codmunres = pd.DataFrame(all_entries, columns=[coluna_codigo, ano, 'Pop'])
del df_pop_fem_10_49_by_codmunres['Pop']

# do a right join
df_pop_fem_10_49_by_codmunres[coluna_codigo] = df_pop_fem_10_49_by_codmunres[coluna_codigo].astype(str)
df_pop_fem_10_49_by_codmunres[ano] = df_pop_fem_10_49_by_codmunres[ano].astype(str)
globals()[which_df][coluna_codigo] = globals()[which_df][coluna_codigo].astype(str)
globals()[which_df][ano] = globals()[which_df][ano].astype(str)
df_merge = pd.merge(globals()[which_df], df_pop_fem_10_49_by_codmunres, on=[coluna_codigo,ano], how='right')
df_merge = df_merge.fillna(0)
globals()[which_df] = df_merge
globals()[which_df][ano] = globals()[which_df][ano].astype(int)
globals()[which_df][coluna_codigo] = globals()[which_df][coluna_codigo].astype(int)

globals()[which_df].head()

CODMUNRES Ano Qtd_Morte_Materna_Direta Qtd_Total Ratio


,CODMUNRES,Ano,Qtd_Morte_Materna_Direta,Qtd_Total,Ratio
0,110001,2012,0.0,386.0,0.0
1,110001,2013,0.0,382.0,0.0
2,110001,2014,0.0,341.0,0.0
3,110001,2015,0.0,403.0,0.0
4,110001,2016,0.0,331.0,0.0


In [179]:
df_mean = globals()[which_df].groupby(['CODMUNRES']).mean()
del df_mean['Ano']
del df_mean['Ratio']
df_mean=df_mean.reset_index(level=0) 
df_mean.head()

,CODMUNRES,Qtd_Morte_Materna,Qtd_Total
0,110001,0.125,374.000
1,110002,1.250,1629.375
2,110003,0.000,75.875
3,110004,0.250,1351.875
4,110005,0.125,258.250


In [185]:
#df_rmm_mean=df_rmm_mean.reset_index(level=0) 
#print(df_rmm_mean.columns.tolist())
print(globals()[which_df].loc[(globals()[which_df]['CODMUNRES'].isin([530010]))].head(10))
#print(df_rmm_mean.loc[(df_rmm_mean['CODMUNRES'].isin([330455]))].head())

       CODMUNRES   Ano  Qtd_Morte_Materna_Direta  Qtd_Total      Ratio
44552     530010  2012                      13.0    43497.0  29.887119
44553     530010  2013                      21.0    44530.0  47.159219
44554     530010  2014                      14.0    44721.0  31.305203
44555     530010  2015                      10.0    46427.0  21.539191
44556     530010  2016                      17.0    43872.0  38.749088
44557     530010  2017                      14.0    44773.0  31.268845
44558     530010  2018                      19.0    44185.0  43.001018
44559     530010  2019                       3.0    42373.0   7.079980


In [172]:
df_menos_20 = df_mean.loc[df_mean['Qtd_Total'] < 20]
df_20_a_99 = df_mean.loc[(df_mean['Qtd_Total'] >= 20) & (df_mean['Qtd_Total'] < 100)]
df_100_a_999 = df_mean.loc[(df_mean['Qtd_Total'] >= 100) & (df_mean['Qtd_Total'] < 1000)]
df_1000_a_4499 = df_mean.loc[(df_mean['Qtd_Total'] >= 1000) & (df_mean['Qtd_Total'] < 4500)]
df_4500_a_9999 = df_mean.loc[(df_mean['Qtd_Total'] >= 4500) & (df_mean['Qtd_Total'] < 10000)]
df_10000_a_50000 = df_mean.loc[(df_mean['Qtd_Total'] >= 10000) & (df_mean['Qtd_Total'] <= 50000)]
df_mais_50000 = df_mean.loc[df_mean['Qtd_Total'] > 50000]
print('menos de 20 =  '+str(len(df_menos_20.index)))
print('entre 20 a 99 =  '+str(len(df_20_a_99.index)))
print('entre 100 a 999 =  '+str(len(df_100_a_999.index)))
print('entre 1000 a 4499 =  '+str(len(df_1000_a_4499.index)))
print('entre 4500 a 9999 =  '+str(len(df_4500_a_9999.index)))
print('entre 10000 a 50000 =  '+str(len(df_10000_a_50000.index)))
print('mais de 50000 =  '+str(len(df_mais_50000.index)))

menos de 20 =  121
entre 20 a 99 =  1987
entre 100 a 999 =  2977
entre 1000 a 4499 =  396
entre 4500 a 9999 =  63
entre 10000 a 50000 =  24
mais de 50000 =  2


In [175]:
coll = 'Qtd_Morte_Materna'

print('Menos de 20')
print(df_menos_20.sort_values([coll], ascending=[False]).head(3))
print('Entre 20 a 99')
print(df_20_a_99.sort_values([coll], ascending=[False]).head(3))
print('Entre 100 a 999')
print(df_100_a_999.sort_values([coll], ascending=[False]).head(3))
print('Entre 1000 a 4499')
print(df_1000_a_4499.sort_values([coll], ascending=[False]).head(3))
print('Entre 4500 a 9999')
print(df_4500_a_9999.sort_values([coll], ascending=[False]).head(3))
print('Entre 10000 a 50000')
print(df_10000_a_50000.sort_values([coll], ascending=[False]).head(3))
print('Mais de 50000')
print(df_mais_50000.sort_values([coll], ascending=[False]).head(3))

Menos de 20
      CODMUNRES  Qtd_Morte_Materna_Direta  Qtd_Total
2776     314570                     0.125     15.625
4651     430222                     0.125     18.500
4911     431413                     0.125     18.625
Entre 20 a 99
      CODMUNRES  Qtd_Morte_Materna_Direta  Qtd_Total
710      220211                      0.25     62.875
4824     431085                      0.25     35.875
2348     310960                      0.25     38.500
Entre 100 a 999
     CODMUNRES  Qtd_Morte_Materna_Direta  Qtd_Total
221     150450                      1.50     612.75
660     211260                      1.25     636.00
233     150520                      1.25     720.25
Entre 1000 a 4499
      CODMUNRES  Qtd_Morte_Materna_Direta  Qtd_Total
1652     270030                     2.875   3982.750
2044     291840                     2.750   4195.750
3851     355250                     2.625   4392.875
Entre 4500 a 9999
     CODMUNRES  Qtd_Morte_Materna_Direta  Qtd_Total
303     160030            

In [176]:
df_mean.sort_values(['Qtd_Total'], ascending=[False]).head(10)

,CODMUNRES,Qtd_Morte_Materna_Direta,Qtd_Total
3829,355030,44.250,170897.875
3242,330455,35.000,86288.125
5569,530010,13.875,44297.250
111,130260,14.625,40512.125
949,230440,11.000,37261.750
2162,292740,10.625,35596.625
2309,310620,7.375,30856.375
4005,410690,3.375,23725.125
1596,261160,6.375,22925.625
5417,520870,6.375,21397.625
